In [1]:
import torch
import numpy as np
import pandas as pd

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


Torch version: 2.1.1+cu121
CUDA available: True
GPU: Quadro P5000


In [2]:
from pathlib import Path

DATA_PATH = Path("artifacts/pems_graph_dataset_strict.npz")
assert DATA_PATH.exists(), f"Missing dataset: {DATA_PATH}"

data = np.load(DATA_PATH, allow_pickle=True)
print("Loaded:", DATA_PATH)
print("Keys:", list(data.keys()))

X = data["X"].astype(np.float32)          # (T,N,F)
Y = data["Y"].astype(np.float32)          # (T,N)
A = data["A"].astype(np.float32)          # (N,N)

stations = data["stations"]
timestamps = pd.to_datetime(data["timestamps"])

train_starts = data["train_starts"]
val_starts   = data["val_starts"]
test_starts  = data["test_starts"]

IN_LEN  = int(data["in_len"])
OUT_LEN = int(data["out_len"])

flow_mean  = data["flow_mean"]
flow_std   = data["flow_std"]
speed_mean = data["speed_mean"]
speed_std  = data["speed_std"]

T, N, Fdim = X.shape
print("X:", X.shape, "Y:", Y.shape)
print("A:", A.shape)
print("IN_LEN:", IN_LEN, "OUT_LEN:", OUT_LEN)
print("Stations:", len(stations))
print("Time range:", timestamps.min(), "→", timestamps.max())


Loaded: artifacts/pems_graph_dataset_strict.npz
Keys: ['X', 'Y', 'A', 'stations', 'timestamps', 'train_starts', 'val_starts', 'test_starts', 'in_len', 'out_len', 'flow_mean', 'flow_std', 'speed_mean', 'speed_std']
X: (2208, 1821, 6) Y: (2208, 1821)
A: (1821, 1821)
IN_LEN: 24 OUT_LEN: 72
Stations: 1821
Time range: 2024-10-01 00:00:00 → 2024-12-31 23:00:00


/tmp/ipykernel_134/1765687619.py:21: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  IN_LEN  = int(data["in_len"])
/tmp/ipykernel_134/1765687619.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  OUT_LEN = int(data["out_len"])


In [3]:
def scaled_laplacian(A):
    A = np.maximum(A, A.T)                     # undirected
    A = A + np.eye(A.shape[0], dtype=np.float32)

    d = A.sum(axis=1)
    d_inv_sqrt = np.power(d, -0.5, where=(d > 0))
    d_inv_sqrt[~np.isfinite(d_inv_sqrt)] = 0.0

    A_norm = (d_inv_sqrt[:, None] * A) * d_inv_sqrt[None, :]
    L = np.eye(A.shape[0], dtype=np.float32) - A_norm

    lambda_max = 2.0
    L_tilde = (2.0 / lambda_max) * L - np.eye(A.shape[0], dtype=np.float32)
    return L_tilde

def dense_to_sparse(A_dense, device):
    idx = np.nonzero(A_dense)
    indices = torch.tensor(np.vstack(idx), dtype=torch.long)
    values = torch.tensor(A_dense[idx], dtype=torch.float32)
    return torch.sparse_coo_tensor(
        indices, values, size=A_dense.shape, device=device
    ).coalesce()

L_tilde = scaled_laplacian(A)
L_sp = dense_to_sparse(L_tilde, DEVICE)

print("L_sp nnz:", int(L_sp._nnz()))


L_sp nnz: 7856


In [4]:
class STGCNDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y, starts, in_len, out_len,
                 flow_mean, flow_std, speed_mean, speed_std):
        self.X = X
        self.Y = Y
        self.starts = starts.astype(int)
        self.in_len = in_len
        self.out_len = out_len
        self.flow_mean = flow_mean
        self.flow_std = flow_std
        self.speed_mean = speed_mean
        self.speed_std = speed_std

    def __len__(self):
        return len(self.starts)

    def __getitem__(self, idx):
        t = self.starts[idx]

        x = self.X[t:t+self.in_len].copy()      # (IN,N,F)
        y = self.Y[t+self.in_len:t+self.in_len+self.out_len].copy()

        # scale
        x[:, :, 0] = (x[:, :, 0] - self.flow_mean) / self.flow_std
        x[:, :, 1] = (x[:, :, 1] - self.speed_mean) / self.speed_std
        y = (y - self.flow_mean) / self.flow_std

        x = np.transpose(x, (2,1,0))            # (F,N,IN)

        return (
            torch.tensor(x, dtype=torch.float32),
            torch.tensor(y, dtype=torch.float32)
        )

train_ds = STGCNDataset(X, Y, train_starts, IN_LEN, OUT_LEN,
                         flow_mean, flow_std, speed_mean, speed_std)
val_ds   = STGCNDataset(X, Y, val_starts, IN_LEN, OUT_LEN,
                         flow_mean, flow_std, speed_mean, speed_std)
test_ds  = STGCNDataset(X, Y, test_starts, IN_LEN, OUT_LEN,
                         flow_mean, flow_std, speed_mean, speed_std)

BATCH_SIZE = 8

train_loader = torch.utils.data.DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=0, pin_memory=False
)
val_loader = torch.utils.data.DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=0, pin_memory=False
)
test_loader = torch.utils.data.DataLoader(
    test_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=0, pin_memory=False
)

xb, yb = next(iter(train_loader))
print("Batch x:", xb.shape, "Batch y:", yb.shape)


Batch x: torch.Size([8, 6, 1821, 24]) Batch y: torch.Size([8, 72, 1821])
